In [1]:
from enum import Enum
import numpy as np
# from usefulFunctions import *
import copy
import  pickle

In [2]:
class Status(Enum):
    STAGED = 200
    ENQUEUED = 201
    DONE = 202


In [3]:

class Task:
    # __actionDict__ = {0:np.array([0,0]), 1:np.array([0,1]), 2:np.array([1,0]), 3:np.array([0,-1]), 4:np.array([-1,0])}
    __actionDict__ = {0:np.array([0,0]), 1:np.array([1,0]), 2:np.array([0,1]), 3:np.array([-1,0]), 4:np.array([0,-1])}

    def __init__(self, tid, rid, start, action, time) -> None:
        self.taskID = tid
        self.robotID = rid
        self.action = action
        self.startPos = np.array(start)
        self.goalPos = np.array(self.__actionDict__[action]+start)
        self.time = time
        self.preqs = dict()
        self.status = Status.STAGED
        
    def __checkIfReady__(self):
        # print("Check if ready", self.preqs)
        if self.taskID-1 not in self.preqs:
            temp = list(self.preqs.values())
            if(np.array_equal(temp, np.full_like(temp, Status.DONE.value))):
                return True
            else:
                return False
        if self.preqs[self.taskID-1]>=Status.ENQUEUED.value:
            temp = copy.deepcopy(self.preqs)
            del temp[self.taskID-1]
            temp = list(temp.values())
            if (np.array_equal(temp, np.full_like(temp, Status.DONE.value))):
                return True
            else:
                return False
        return False
    
    def __publishTask__(self):
        global TOPIC
        message = ROS_Task()
        message.taskID = self.taskID
        message.robotID = self.robotID
        message.action = self.action
        message.status = self.status.value

        TOPIC.publish(message)

    
    def __repr__(self) -> str:
        asd = "\n"
        for i in self.__dir__():
            if not i.startswith('__'):
                asd+=i
                asd+=": "
                asd+=str(getattr(self,i))
                asd+=", "

        return asd

class Graph:
    def __init__(self, taskList, startCells) -> None:
        self.currentPositions = copy.deepcopy(startCells)
        self.graph = dict()
        self.taskDict = dict()
        tId = 1
        self.robotList = []
        for rid in range(len(startCells)):
        
            prevTask = None
            for i, task in enumerate(taskList[:,rid]):
                t = Task(tId, rid, self.currentPositions[rid], task, i)
                # print(t)
                self.taskDict[tId] = t
                tId+=1
                self.currentPositions[rid] = t.goalPos
                self.addVertex(t)

                if(prevTask is None):
                    self.robotList.append(t)
                else:
                    self.addEdge(prevTask, t)

                prevTask = t
        # print(taskDict)
        for rid in range(len(startCells)):
            firstTid = self.robotList[rid].taskID
            for taskID in range(firstTid, firstTid+len(taskList)):
                if(taskID not in self.taskDict):
                    continue
                task = self.taskDict[taskID]
                
                for rid_ in range(len(startCells)):
                    if(rid != rid_):
                        # print(rid, rid_)
                        firstTid_ = self.robotList[rid_].taskID
                        for taskID_ in range(firstTid_, firstTid_+len(taskList)):
                            if(taskID_ not in self.taskDict):
                                continue
                            task_ = self.taskDict[taskID_]
                            if np.array_equal(task.startPos, task_.goalPos) and task.time<=task_.time:
                                # print(task, task_)
                                self.addEdge(task, task_)
                                break

            
    
    def addVertex(self, task):
        assert type(task) == Task
        if(task.taskID not in self.graph):
            self.graph[task.taskID] = list()

    def addEdge(self, taskFrom, taskTo):
        assert  type(taskFrom)==Task and taskFrom.taskID in self.graph
        assert type(taskTo)==Task and taskTo.taskID in self.graph
        taskTo.preqs[taskFrom.taskID] = taskFrom.status.value
        self.graph[taskFrom.taskID].append(taskTo.taskID)

    def changeStatus(self, taskID):
        # rospy.loginfo(taskID)
        task = self.taskDict[taskID]
        # print(taskID, task)
        task.status = Status(task.status.value+1)

        task.__publishTask__()

In [4]:
actionList= np.loadtxt('actions.txt')
actionList = actionList.astype(np.int16)
START=[(2, 7),(20, 1),(17, 16),(2, 0),(10, 1),(16, 22),(17, 7),(1, 4),(5, 5),(17, 18),(7, 22),(21, 20),(7, 1),(1, 2),(15, 21),(2, 17),(5, 10),(17, 13),(10, 22),(2, 21),(8, 4),(8, 19),(5, 3),(20, 3),(14, 12),(19, 20),(1, 7),(2, 14),(14, 21),(15, 2),(11, 21),(17, 1),(2, 10),(5, 9),(1, 5),(13, 1),(22, 2),(15, 14),(3, 21),(2, 8),(1, 0),(22, 16),(0, 14),(21, 2),(8, 22),(18, 8),(18, 1),(0, 20),(1, 22),(17, 5),(11, 8),(17, 11),(8, 17),(0, 9),(16, 0),(17, 2),(4, 8),(14, 19),(2, 11),(12, 14),(8, 6),(21, 1),(2, 16),(21, 17),(0, 18),(19, 22),(20, 16),(9, 2),(0, 17),(7, 8),(1, 6),(11, 0),(17, 3),(0, 0),(9, 14),(22, 10),(8, 9),(21, 16),(21, 15),(1, 1),(20, 15),(19, 1),(9, 22),(22, 3),(2, 9),(7, 21),(14, 13),(0, 21),(12, 8),(0, 2),(18, 20),(17, 22),(1, 10),(0, 10),(5, 4),(22, 5),(13, 8),(2, 22),(5, 15),(6, 14),(1, 13),(5, 19),(22, 7),(0, 12),(15, 22),(21, 19),(5, 20),(10, 8),(21, 7),(22, 0),(7, 2),(1, 16),(11, 19),(14, 8),(8, 5),(1, 19),(1, 11),(0, 5),(1, 20),(8, 7),(5, 18),(14, 6),(22, 21),(14, 17),(6, 2),(18, 14),(15, 20),(12, 22),(21, 12),(6, 8),(0, 7),(11, 17),(22, 20),(0, 11),(11, 6),(14, 2),(2, 3),(16, 2),(14, 10),(0, 13),(16, 1),(15, 1),(21, 21),(18, 21),(0, 4),(5, 22),(20, 6),(1, 9),(20, 14),(12, 0),(0, 8),(20, 5),(20, 13),(12, 2),(13, 22),(1, 3),(21, 10),(11, 3),(4, 1),(17, 21),(20, 18),(14, 3),(14, 14),(0, 6),(7, 14),(22, 8),(8, 1),(2, 19),(12, 1),(20, 4),(18, 0),(1, 17),(5, 1),(22, 18),(19, 0),(22, 9),(9, 8),(17, 17),(2, 12),(21, 8),(8, 21),(20, 19),(5, 11),(17, 10),(17, 6),(22, 14),(3, 20),(11, 15),(3, 2),(8, 3),(17, 15),(13, 21),(18, 22),(14, 1),(21, 3),(6, 0),(8, 8),(14, 11),(21, 13),(11, 1),(10, 20),(22, 13),(5, 13),(6, 21),(0, 22),(3, 22),(0, 16),(3, 14),(4, 14),(1, 15),(5, 14),(8, 15)]
START = START[:30]
actionList = actionList[:,:30]


In [5]:
GRAPH = Graph(actionList, START)

In [6]:

class GraphTD:
    def __init__(self, taskList, startCells) -> None:
        positions = {}
        self.currentPositions = copy.deepcopy(startCells)

        self.graph = dict()
        self.taskDict = dict()
        tId = 1
        self.robotList = []
        for rid in range(len(startCells)):
        
            prevTask = None
            for i, task in enumerate(taskList[:,rid]):

                # print(tId, rid, currentPositions[rid], task, i)
                t = Task(tId, rid, self.currentPositions[rid], task, i)
                # print(t)
                self.taskDict[tId] = t
                tId+=1
                self.currentPositions[rid] = t.goalPos
                self.addVertex(t)

                if(tuple(t.goalPos) not in positions):
                    positions[tuple(t.goalPos)] = dict()
                positions[tuple(t.goalPos)][t.time] = t.taskID

                if(prevTask is None):
                    self.robotList.append(t)
                else:
                    self.addEdge(prevTask, t)

                prevTask = t
        # print(self.taskDict)

        # self.pos = positions
        for pos in positions:
            timeList = sorted(positions[pos].keys())
            # print(timeList)
            # print(positions[pos][0])
            for i in range(len(timeList)):
                for j in range(i+1, len(timeList)):
                    # print(positions[pos], positions[pos][timeList[i]], positions[pos][timeList[j]])
                    self.addEdge(self.taskDict[positions[pos][timeList[i]]+1], self.taskDict[positions[pos][timeList[j]]])

        for t in range(1, len(self.taskDict), len(taskList)):
            # print(self.taskDict[t])
            if(tuple(self.taskDict[t].startPos) in positions):
                for time in positions[tuple(self.taskDict[t].startPos)]:
                    self.addEdge(self.taskDict[t], self.taskDict[positions[tuple(self.taskDict[t].startPos)][time]])



        for rid in range(len(startCells)):
            firstTid = self.robotList[rid].taskID
            for taskID in range(firstTid, firstTid+len(taskList)):
                if(taskID not in self.taskDict):
                    continue
                task = self.taskDict[taskID]
                
        #         for rid_ in range(len(startCells)):
        #             if(rid == rid_):
        #                 continue
        #             else:
        #                 firstTid_ = self.robotList[rid_].taskID
        #                 for taskID_ in range(firstTid_, firstTid_+len(taskList)):
        #                     if(taskID_ not in self.taskDict):
        #                         continue
        #                     task_ = self.taskDict[taskID_]
        #                     if np.array_equal(task.startPos, task_.goalPos) and task.time<=task_.time:
        #                         self.addEdge(task, task_)
        #                         break

            
    
    def addVertex(self, task):
        assert type(task) == Task
        if(task.taskID not in self.graph):
            self.graph[task.taskID] = list()


    def addEdge(self, taskFrom, taskTo):
        assert  type(taskFrom)==Task and taskFrom.taskID in self.graph
        assert type(taskTo)==Task and taskTo.taskID in self.graph

        if(taskFrom.taskID in taskTo.preqs or taskFrom.taskID==taskTo.taskID):
            return
        for t in self.graph[taskFrom.taskID]:
            if(self.taskDict[t].robotID==taskTo.robotID):
                return


        taskTo.preqs[taskFrom.taskID] = taskFrom.status.value
        self.graph[taskFrom.taskID].append(taskTo.taskID)

    def changeStatus(self, taskID):
        # rospy.loginfo(taskID)
        task = self.taskDict[taskID]
        # print(taskID, task)
        task.status = Status(task.status.value+1)

        task.__publishTask__()

In [7]:
G2 = GraphTD(actionList, START)
# G2.taskDict


In [8]:
asd = G2.currentPositions
GOALS = [(9, 0),(0, 19),(11, 2),(0, 6),(5, 0),(11, 4),(8, 6),(14, 21),(22, 3),(11, 14),(20, 14),(0, 5),(15, 8),(17, 12),(0, 0),(19, 14),(1, 22),(0, 11),(3, 8),(22, 15),(5, 22),(21, 8),(17, 11),(3, 21),(5, 12),(14, 1),(17, 6),(5, 4),(1, 19),(17, 18),(14, 13),(17, 0),(2, 21),(11, 11),(4, 21),(5, 13),(11, 5),(11, 9),(8, 4),(20, 2),(16, 2),(17, 16),(5, 17),(0, 8),(8, 16),(6, 1),(14, 20),(22, 6),(22, 1),(4, 22),(5, 14),(6, 8),(20, 8),(11, 16),(22, 13),(0, 22),(11, 20),(20, 18),(17, 5),(10, 20),(22, 22),(22, 0),(8, 8),(5, 10),(21, 16),(22, 12),(17, 9),(12, 1),(21, 5),(14, 2),(22, 2),(13, 0),(8, 18),(21, 6),(21, 0),(20, 9),(10, 22),(21, 1),(20, 22),(10, 1),(12, 8),(7, 8),(0, 10),(13, 21),(1, 3),(8, 13),(22, 17),(21, 2),(18, 8),(11, 8),(17, 2),(18, 1),(19, 21),(19, 22),(5, 5),(15, 0),(10, 21),(16, 20),(5, 21),(8, 5),(13, 8),(0, 15),(22, 8),(13, 20),(21, 3),(2, 12),(22, 20),(5, 19),(10, 8),(4, 0),(1, 9),(16, 22),(2, 0),(5, 1),(5, 16),(15, 20),(0, 12),(0, 1),(21, 21),(14, 18),(2, 1),(14, 4),(14, 5),(1, 17),(17, 13),(6, 14),(18, 0),(17, 3),(10, 2),(2, 13),(22, 21),(4, 1),(1, 20),(17, 21),(0, 17),(20, 6),(1, 2),(4, 8),(14, 9),(14, 10),(5, 20),(14, 11),(14, 22),(3, 22),(22, 9),(12, 2),(11, 1),(22, 7),(4, 14),(22, 14),(0, 13),(17, 22),(0, 3),(20, 21),(2, 7),(1, 8),(21, 14),(11, 13),(10, 0),(15, 1),(11, 19),(22, 11),(11, 18),(6, 2),(19, 1),(21, 4),(0, 4),(3, 2),(18, 20),(5, 6),(15, 21),(20, 0),(1, 14),(2, 15),(21, 19),(20, 11),(11, 22),(17, 19),(16, 0),(22, 10),(4, 20),(13, 1),(20, 16),(22, 4),(8, 12),(9, 21),(8, 1),(7, 14),(12, 21),(17, 7),(8, 2),(2, 4),(14, 8),(20, 1),(21, 11),(3, 1),(18, 2),(9, 8),(5, 7),(21, 17),(8, 0),(0, 18),(8, 20),(2, 14),(8, 14),(1, 21),(0, 14),(1, 13),(2, 19),(1, 16),(6, 21),(1, 18)]
GOALS = GOALS[:6]

for i in range(len(GOALS)):
    if(np.array_equal(asd[i], GOALS[i])):
        pass
    else:
        print(False)

In [9]:
for i in GRAPH.graph:
    if not (np.array_equal(set(GRAPH.graph[i]), set(G2.graph[i]))):
        print(i, GRAPH.graph[i], G2.graph[i])
    for j in GRAPH.graph[i]:
        if(j not in G2.graph[i]):
            print("False")

In [37]:
for i in G2.graph:
    G2.graph[i] = sorted(G2.graph[i])

for i in GRAPH.graph:
    GRAPH.graph[i] = sorted(GRAPH.graph[i])
    

In [65]:
for i in G2.taskDict:
    if not(G2.taskDict[i].action == GRAPH.taskDict[i].action):
        print(False)
        

In [35]:
i = 3
for j in G2.graph[i]:
    print(j)
    if(j not in GRAPH.graph[i]):
        
        print("False")

4
1696


In [14]:
for i in G2.graph:
    # print(set(GRAPH.graph[i]), set(G2.graph[i]))
    if not (np.array_equal(set(GRAPH.graph[i]), set(G2.graph[i]))):
        print(i, GRAPH.graph[i], G2.graph[i])
    for j in G2.graph[i]:
        if(j not in GRAPH.graph[i]):
            print("False")

In [38]:
# import json 
# with open("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/pickle_test.txt", "w") as f:
#     f.write(json.dumps(GRAPH.graph))

In [36]:
# import json 
# with open("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/TD_test.txt", "w") as f:
#     f.write(json.dumps(G2.graph))

In [55]:
# with open("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/pickle_test.obj", "r") as f:
#     G3 = pickle.load(f)

temp = {}
for i in GRAPH.graph:
    temp[i] = {}
    for j in dir(GRAPH.taskDict[i]):
        # print(j)
        if not j.startswith("__"):
            temp[i][j] = getattr(GRAPH.taskDict[i], j)

In [58]:
temp

{1: {'action': 0,
  'goalPos': array([2, 7]),
  'preqs': {},
  'robotID': 0,
  'startPos': array([2, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 1,
  'time': 0},
 2: {'action': 0,
  'goalPos': array([2, 7]),
  'preqs': {1: 200},
  'robotID': 0,
  'startPos': array([2, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 2,
  'time': 1},
 3: {'action': 3,
  'goalPos': array([1, 7]),
  'preqs': {2: 200, 1613: 200},
  'robotID': 0,
  'startPos': array([2, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 3,
  'time': 2},
 4: {'action': 0,
  'goalPos': array([1, 7]),
  'preqs': {3: 200},
  'robotID': 0,
  'startPos': array([1, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 4,
  'time': 3},
 5: {'action': 0,
  'goalPos': array([1, 7]),
  'preqs': {4: 200},
  'robotID': 0,
  'startPos': array([1, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 5,
  'time': 4},
 6: {'action': 0,
  'goalPos': array([1, 7]),
  'preqs': {5: 200},
  'robotID': 0,
  'startPos': array([1, 7]),
  'statu

In [59]:
temp2 = {}
for i in G2.graph:
    temp2[i] = {}
    for j in dir(G2.taskDict[i]):
        # print(j)
        if not j.startswith("__"):
            temp2[i][j] = getattr(G2.taskDict[i], j)

In [60]:
temp2

{1: {'action': 0,
  'goalPos': array([2, 7]),
  'preqs': {},
  'robotID': 0,
  'startPos': array([2, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 1,
  'time': 0},
 2: {'action': 0,
  'goalPos': array([2, 7]),
  'preqs': {1: 200},
  'robotID': 0,
  'startPos': array([2, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 2,
  'time': 1},
 3: {'action': 3,
  'goalPos': array([1, 7]),
  'preqs': {2: 200, 1613: 200},
  'robotID': 0,
  'startPos': array([2, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 3,
  'time': 2},
 4: {'action': 0,
  'goalPos': array([1, 7]),
  'preqs': {3: 200},
  'robotID': 0,
  'startPos': array([1, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 4,
  'time': 3},
 5: {'action': 0,
  'goalPos': array([1, 7]),
  'preqs': {4: 200},
  'robotID': 0,
  'startPos': array([1, 7]),
  'status': <Status.STAGED: 200>,
  'taskID': 5,
  'time': 4},
 6: {'action': 0,
  'goalPos': array([1, 7]),
  'preqs': {5: 200},
  'robotID': 0,
  'startPos': array([1, 7]),
  'statu

In [74]:
temp = {}
for i in GRAPH.graph:
    temp[i] = {}
    temp[i]["action"] = int(GRAPH.taskDict[i].action)
    temp[i]["preqs"] = list(GRAPH.taskDict[i].preqs.keys())
    temp[i]["robotID"] = GRAPH.taskDict[i].robotID

In [75]:
temp

{1: {'action': 0, 'preqs': [], 'robotID': 0},
 2: {'action': 0, 'preqs': [1], 'robotID': 0},
 3: {'action': 3, 'preqs': [2, 1613], 'robotID': 0},
 4: {'action': 0, 'preqs': [3], 'robotID': 0},
 5: {'action': 0, 'preqs': [4], 'robotID': 0},
 6: {'action': 0, 'preqs': [5], 'robotID': 0},
 7: {'action': 0, 'preqs': [6], 'robotID': 0},
 8: {'action': 2, 'preqs': [7], 'robotID': 0},
 9: {'action': 1, 'preqs': [8], 'robotID': 0},
 10: {'action': 0, 'preqs': [9], 'robotID': 0},
 11: {'action': 0, 'preqs': [10], 'robotID': 0},
 12: {'action': 3, 'preqs': [11], 'robotID': 0},
 13: {'action': 1, 'preqs': [12], 'robotID': 0},
 14: {'action': 4, 'preqs': [13], 'robotID': 0},
 15: {'action': 4, 'preqs': [14], 'robotID': 0},
 16: {'action': 4, 'preqs': [15], 'robotID': 0},
 17: {'action': 4, 'preqs': [16, 1617], 'robotID': 0},
 18: {'action': 0, 'preqs': [17], 'robotID': 0},
 19: {'action': 4, 'preqs': [18, 437, 1618], 'robotID': 0},
 20: {'action': 3, 'preqs': [19, 436], 'robotID': 0},
 21: {'actio

In [76]:
with open("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/pickle_taskList.txt", "w") as f:
    f.write(json.dumps(temp))

In [77]:
import json
 
# Opening JSON file
with open("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/pickle_taskList.txt") as f:
    taskList = json.load(f)